# Logistic Regression

In [73]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import math
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import statsmodels.api as sm

In [74]:
# Importing dataset
data = pd.read_csv("FinalR.csv")

In [76]:
data.head()

,Severity,City,State,Temperature(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Speed(mph),Amenity,Bump,...,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight,year,Date,Month,Start Time,End Time
0,3,Westfield,NY,26.1,66,30.19,10.0,3.5,0,0,...,0,1,1,1,1,2019,19/03/03,3,9:53:41,10:22:21
1,3,Hamburg,NY,28.0,61,30.03,10.0,5.8,0,0,...,0,1,1,1,1,2019,19/03/03,3,15:25:49,15:55:22
2,3,Grand Island,NY,25.0,75,30.04,2.5,8.1,0,0,...,0,1,1,1,1,2019,19/03/03,3,17:36:46,18:06:29
3,3,Niagara Falls,NY,25.0,75,30.04,2.5,8.1,0,0,...,0,1,1,1,1,2019,19/03/03,3,17:48:42,18:18:20
4,2,Bradford,PA,24.1,88,29.95,0.5,5.8,0,0,...,0,1,1,1,1,2019,19/03/03,3,17:54:31,18:24:07


In [78]:
data = data.drop(["City","Start Time","End Time"],axis = 1)

In [79]:
data.head()

,Severity,State,Temperature(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Speed(mph),Amenity,Bump,Crossing,...,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight,year,Date,Month
0,3,NY,26.1,66,30.19,10.0,3.5,0,0,0,...,0,0,0,1,1,1,1,2019,19/03/03,3
1,3,NY,28.0,61,30.03,10.0,5.8,0,0,0,...,0,0,0,1,1,1,1,2019,19/03/03,3
2,3,NY,25.0,75,30.04,2.5,8.1,0,0,0,...,0,0,0,1,1,1,1,2019,19/03/03,3
3,3,NY,25.0,75,30.04,2.5,8.1,0,0,0,...,0,0,0,1,1,1,1,2019,19/03/03,3
4,2,PA,24.1,88,29.95,0.5,5.8,0,0,0,...,0,0,0,1,1,1,1,2019,19/03/03,3


In [80]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223223 entries, 0 to 223222
Data columns (total 27 columns):
Severity                 223223 non-null int64
State                    223223 non-null object
Temperature(F)           223223 non-null float64
Humidity(%)              223223 non-null int64
Pressure(in)             223223 non-null float64
Visibility(mi)           223223 non-null float64
Wind_Speed(mph)          223223 non-null float64
Amenity                  223223 non-null int64
Bump                     223223 non-null int64
Crossing                 223223 non-null int64
Give_Way                 223223 non-null int64
Junction                 223223 non-null int64
No_Exit                  223223 non-null int64
Railway                  223223 non-null int64
Roundabout               223223 non-null int64
Station                  223223 non-null int64
Stop                     223223 non-null int64
Traffic_Calming          223223 non-null int64
Traffic_Signal           223223 no

In [81]:
data["Temperature(F)"] = data["Temperature(F)"].astype(int)
data["Pressure(in)"] = data["Pressure(in)"].astype(int)
data["Visibility(mi)"] = data["Visibility(mi)"].astype(int)
data["Wind_Speed(mph)"] = data["Wind_Speed(mph)"].astype(int)
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223223 entries, 0 to 223222
Data columns (total 27 columns):
Severity                 223223 non-null int64
State                    223223 non-null object
Temperature(F)           223223 non-null int32
Humidity(%)              223223 non-null int64
Pressure(in)             223223 non-null int32
Visibility(mi)           223223 non-null int32
Wind_Speed(mph)          223223 non-null int32
Amenity                  223223 non-null int64
Bump                     223223 non-null int64
Crossing                 223223 non-null int64
Give_Way                 223223 non-null int64
Junction                 223223 non-null int64
No_Exit                  223223 non-null int64
Railway                  223223 non-null int64
Roundabout               223223 non-null int64
Station                  223223 non-null int64
Stop                     223223 non-null int64
Traffic_Calming          223223 non-null int64
Traffic_Signal           223223 non-null i

In [82]:
# Visulizing the data

In [ ]:
#sb.countplot(x="Severity",hue= "State",orient='h',data=data)

g = sb.FacetGrid(data, col='Severity', hue='State')
g = g.map(sb.swarmplot, 'State', 'Temperature(F)')

## Regression

In [85]:
data.drop(["Date","Humidity(%)","Pressure(in)","Visibility(mi)","Wind_Speed(mph)","Amenity",
          "year"],axis =1,inplace=True)

In [86]:
data.head()

,Severity,State,Temperature(F),Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight,Month
0,3,NY,26,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,3
1,3,NY,28,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,3
2,3,NY,25,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,3
3,3,NY,25,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,3
4,2,PA,24,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,3


In [87]:
Lx_axis = data.drop("Severity",axis=1)
Ly_axis = data["Severity"]

In [88]:
# Splitting the data into 80:20 ratio

Lx_axis_train,Lx_axis_test,Ly_axis_train,Ly_axis_test = train_test_split(Lx_axis,Ly_axis,test_size=0.2,random_state=100)

In [89]:
logit = sm.GLM(LY_train, LX_train, family = sm.families.Binomial())

In [90]:
result1 = logit.fit()

C:\Users\robiy\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\genmod\families\links.py:190: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)
C:\Users\robiy\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:889: RuntimeWarning: divide by zero encountered in true_divide
  endog_mu = self._clean(endog / mu)
C:\Users\robiy\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:890: RuntimeWarning: divide by zero encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))
C:\Users\robiy\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:890: RuntimeWarning: invalid value encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))


In [91]:
print(result1.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:               Severity   No. Observations:               156256
Model:                            GLM   Df Residuals:                   156233
Model Family:                Binomial   Df Model:                           22
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                    nan
Date:                Fri, 17 Jan 2020   Deviance:                          nan
Time:                        15:16:14   Pearson chi2:                 1.52e+21
No. Iterations:                   100                                         
Covariance Type:            nonrobust                                         
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Temperature(F)        -1.896e+

C:\Users\robiy\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:942: RuntimeWarning: divide by zero encountered in true_divide
  special.gammaln(n - y + 1) + y * np.log(mu / (1 - mu)) +
C:\Users\robiy\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:942: RuntimeWarning: invalid value encountered in multiply
  special.gammaln(n - y + 1) + y * np.log(mu / (1 - mu)) +
C:\Users\robiy\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:942: RuntimeWarning: invalid value encountered in add
  special.gammaln(n - y + 1) + y * np.log(mu / (1 - mu)) +
C:\Users\robiy\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:943: RuntimeWarning: divide by zero encountered in log
  n * np.log(1 - mu)) * var_weights
C:\Users\robiy\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\genmod\families\family.py:943: RuntimeWarning: invalid va

In [92]:
np.exp(result1.params)

C:\Users\robiy\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\series.py:856: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)


Temperature(F)           0.000000e+00
Humidity(%)              0.000000e+00
Pressure(in)                      inf
Visibility(mi)           0.000000e+00
Wind_Speed(mph)                   inf
Amenity                  0.000000e+00
Bump                     0.000000e+00
Crossing                 0.000000e+00
Give_Way                          inf
Junction                          inf
No_Exit                           inf
Railway                           inf
Roundabout               0.000000e+00
Station                  0.000000e+00
Stop                     0.000000e+00
Traffic_Calming          0.000000e+00
Traffic_Signal           0.000000e+00
Turning_Loop             8.710730e-42
Sunrise_Sunset                    inf
Civil_Twilight           0.000000e+00
Nautical_Twilight        0.000000e+00
Astronomical_Twilight    0.000000e+00
year                              inf
Month                    0.000000e+00
dtype: float64

In [93]:
result1.deviance

nan

In [94]:
result1.aic

nan

In [95]:
probability = result1.predict(LX_test)

In [96]:
probability

111243    1.0
1507      1.0
62524     1.0
48537     1.0
81299     1.0
         ... 
140888    1.0
211344    1.0
145361    1.0
48138     1.0
191620    1.0
Length: 66967, dtype: float64

In [97]:
auc = roc_auc_score(Ly_axis,probability)
auc

ValueError: multi_class must be in ('ovo', 'ovr')

## 2nd model by dropping more columns

In [104]:
# Importing dataset
data1 = pd.read_csv("FinalR.csv")

In [106]:
data1.drop(["City","Start Time","End Time","Date","Humidity(%)","Pressure(in)","Visibility(mi)","Wind_Speed(mph)","Amenity",
          "year","State","Temperature(F)"],axis =1,inplace=True)

In [107]:
data1.head()

,Severity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight,Month
0,3,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,3
1,3,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,3
2,3,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,3
3,3,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,3
4,2,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,3


In [110]:
Lx_axis1 = data1.drop("Severity",axis=1)
Ly_axis1 = data1["Severity"]

In [111]:
# Splitting the data into 80:20 ratio

Ly1_axis_train,Lx1_axis_test,Ly1_axis_train,Ly1_axis_test = train_test_split(Lx_axis1,Ly_axis1,test_size=0.2,random_state=100)

In [113]:
logit = sm.GLM(Ly1_axis_train,Ly1_axis_train, family=sm.families.Binomial())

In [116]:
result2 = logit.fit()

In [117]:
print(result2.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:               Severity   No. Observations:               178578
Model:                            GLM   Df Residuals:                   178577
Model Family:                Binomial   Df Model:                            0
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                    nan
Date:                Fri, 17 Jan 2020   Deviance:                          nan
Time:                        15:29:59   Pearson chi2:                 1.74e+21
No. Iterations:                   100                                         
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Severity    2.669e+17   6.54e+04   4.08e+12      0.0

In [118]:
np.exp(result2.params)

C:\Users\robiy\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\series.py:856: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)


Severity    inf
dtype: float64

In [119]:
result2.aic

nan